In [ ]:
!sudo apt-get install -y fonts-nanum
!sudo fc-cache -fv
!rm ~/.cache/matplotlib -rf

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-510
Use 'sudo apt autoremove' to remove it.
The following NEW packages will be installed:
  fonts-nanum
0 upgraded, 1 newly installed, 0 to remove and 27 not upgraded.
Need to get 9,599 kB of archives.
After this operation, 29.6 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu focal/universe amd64 fonts-nanum all 20180306-3 [9,599 kB]
Fetched 9,599 kB in 2s (4,327 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 76, <> line 1.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype

In [ ]:
import pandas as pd
import cv2
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
from matplotlib import font_manager
for font in font_manager.fontManager.ttflist:
    if 'Nanum' in font.name:
        print(font.name, font.fname)

In [ ]:
import matplotlib.pyplot as plt

plt.rc('font', family='NanumBarunGothic') 
plt.rcParams['axes.unicode_minus'] = False

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
cropped_img_df = pd.read_csv('/content/drive/MyDrive/공유폴더/angle_cropped_img/angle_cropped_df.csv', index_col = 0)
info_df = pd.read_csv('/content/drive/MyDrive/공유폴더/csv/kcar_info.csv', index_col = 0)
unique_df = pd.read_csv('/content/drive/MyDrive/공유폴더/csv/same_img_Euclidean_Distance.csv', index_col = 0)

# ReidHelper

In [ ]:
!pip install onnx
!pip install onnxoptimizer
!pip install onnxsim
!pip install onnxruntime-gpu
!pip install yacs

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.5/13.5 MB 47.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 78.0 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.19.6
    Uninstalling protobuf-3.19.6:
      Successfully uninstalled protobuf-3.19.6
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.9.2 requires protobuf<3.20,>=3.9.2, but you have protobuf 3.20.3 which is incompatible.
tensorboard 2.9.1 requires protobuf<3.20,>=3.9.2, but you have protobuf 3.20.3 which is incompatible.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 606.4/606.4 KB 6.0 MB/s eta 0:00:00
Looking in indexes: 

In [ ]:
import glob
import os

import cv2
import numpy as np
import onnxruntime

class ReidHelper:

    def __init__(self,model_path):
        self.size = (256,256)
        ep_list = ['CUDAExecutionProvider']
        self.sess = onnxruntime.InferenceSession(model_path,providers = ep_list)

    def infer(self, image_np):
        input_name = self.sess.get_inputs()[0].name

        image = self.preprocess(image_np)

        feat = self.sess.run(None, {input_name: image})[0]
        feat = self.normalize(feat, axis=1)
        
        return feat

    def preprocess(self, image_np):

        # the model expects RGB inputs
        original_image = image_np[:, :, ::-1]

        # Apply pre-processing to image.
        resize_width = self.size[0]
        resize_height = self.size[1]
        img = cv2.resize(original_image, (resize_width, resize_height), interpolation=cv2.INTER_CUBIC)
        img = img.astype("float32").transpose(2, 0, 1)[np.newaxis]  # (1, 3, h, w)
        return img


    def normalize(self, nparray, order=2, axis=-1):
        """Normalize a N-D numpy array along the specified axis."""
        norm = np.linalg.norm(nparray, ord=order, axis=axis, keepdims=True)
        return nparray / (norm + np.finfo(np.float32).eps)

# 거리 계산 함수
def calc_distance(feat1, feat2):
    distance = np.linalg.norm(feat1 - feat2)
    return distance

# Test

In [ ]:
import random
import pandas as pd
import random
import cv2


def random_df(angle = '정면', path_df = cropped_img_df, unique_df = unique_df):
    
    path_dict = {'small':'/content/drive/MyDrive/공유폴더/fast-reid/outputs/VeRi_Baseline/baseline_R50.onnx',
                'middel':'/content/drive/MyDrive/공유폴더/fast-reid/outputs/VehicleID_Baseline/baseline_R50.onnx',
                'large':'/content/drive/MyDrive/공유폴더/fast-reid/outputs/VERI_Wild_Baseline/baseline_R50.onnx'
                 }
    
    idx = 0
    result_df = pd.DataFrame(columns = ['case', 'target_model', 'car_model', 'small_distance', 'middle_distance', 'large_distance', 'file_path'])
    case_idx = 0
    num = 0

    angle_df = path_df[path_df['angle'] == angle].reset_index(drop=True)
    path_v = angle_df['class'].value_counts()
    cls_over5= path_v[path_v>=5].keys()
    
    helper_small = ReidHelper(path_dict['small'])
    helper_middle = ReidHelper(path_dict['middle'])
    helper_large = ReidHelper(path_dict['large'])
    

    for cls in cls_over5:
        select_path = angle_df[angle_df['class']== cls]['file_path'].values
        select_length = len(select_path)
        
        feat1_small = None
        feat1_middle = None
        feat1_large = None

        for index, file_path in enumerate(select_path):
            car_img = cv2.imread(file_path)
            feat_small = helper_small.infer(car_img)
            feat_middle = helper_middle.infer(car_img)
            feat_large = helper_large.infer(car_img)

            if index == 0:
                idx +=1
                feat1_small = feat_small
                feat1_middle =feat_middle
                feat1_large = feat_large
                ex = angle_df[angle_df['class']==cls].iloc[0]
                result_df.loc[idx,:] = [case_idx, cls, cls, 0, 0, 0,  file_path]
                num += 1
            else:
                idx +=1
                small_distance = calc_distance(feat1_small, feat_small)
                middle_distance = calc_distance(feat1_middle, feat_middle)
                large_distance = calc_distance(feat1_large, feat_large)
                result_df.loc[idx,:] = [case_idx, cls, cls, small_distance, middle_distance, large_distance,  file_path]
                num += 1
        
        select_can = list(angle_df[angle_df['class'] != cls].index)
        
        random100 = random.sample(select_can, 100 - select_length)

        for gal_index in random100:
            idx += 1
            gal = angle_df.loc[gal_index]
            car_img = cv2.imread(gal['file_path'])
            feat_small = helper_small.infer(car_img)
            feat_middle = helper_middle.infer(car_img)
            feat_large = helper_large.infer(car_img)
            small_distance = calc_distance(feat1_small, feat_small)
            middle_distance = calc_distance(feat1_middle, feat_middle)
            large_distance = calc_distance(feat1_large, feat_large)
            result_df.loc[idx,:] = [case_idx, cls, gal['class'], small_distance, middle_distance, large_distance,  gal['file_path']]
            num += 1
        
        print(num)
    
    return result_df

In [ ]:
df = random_df()
df.to_csv('/content/drive/MyDrive/공유폴더/csv/정면.csv')
df = random_df(angle = '후면')
df.to_csv('/content/drive/MyDrive/공유폴더/csv/후면.csv')
df = random_df(angle = '측면')
df.to_csv('/content/drive/MyDrive/공유폴더/csv/측면.csv')

KeyboardInterrupt: ignored

In [ ]:
import random
import pandas as pd
import random
import cv2

def random_df(angle = '정면', sample_num=100, path_df = cropped_img_df, unique_df = unique_df):
    
    path_dict = {'small':'/content/drive/MyDrive/공유폴더/fast-reid/outputs/VehicleID_Baseline/baseline_R50.onnx',
                 'middle':'/content/drive/MyDrive/공유폴더/fast-reid/outputs/VeRi_Baseline/baseline_R50.onnx',
                 'large':'/content/drive/MyDrive/공유폴더/fast-reid/outputs/VERI_Wild_Baseline/baseline_R50.onnx'
                 }
    
    idx = 0
    result_df = pd.DataFrame(columns = ['case', 'target_model', 'car_model', 'small_distance', 'middle_distance', 'large_distance', 'file_path'])
    case_idx = 0
    num = 0

    angle_df = path_df[path_df['angle'] == angle].reset_index(drop=True)
    path_v = angle_df['class'].value_counts()
    cls_over5= path_v[path_v>=5].keys()
    
    helper_small = ReidHelper(path_dict['small'])
    helper_middle = ReidHelper(path_dict['middle'])
    helper_large = ReidHelper(path_dict['large'])
    

    for cls in cls_over5:
        select_path = angle_df[angle_df['class']== cls]['file_path'].values
        select_length = len(select_path)
        select_color = angle_df[angle_df['class']== cls]['color'].values[0]

        feat1_small = None
        feat1_middle = None
        feat1_large = None

        for index, file_path in enumerate(select_path):
            car_img = cv2.imread(file_path)
            feat_small = helper_small.infer(car_img)
            feat_middle = helper_middle.infer(car_img)
            feat_large = helper_large.infer(car_img)

            if index == 0:
                idx +=1
                feat1_small = feat_small
                feat1_middle =feat_middle
                feat1_large = feat_large
                ex = angle_df[angle_df['class']==cls].iloc[0]
                result_df.loc[idx,:] = [case_idx, cls, cls, 0, 0, 0,  file_path]
                
            else:
                idx +=1
                small_distance = calc_distance(feat1_small, feat_small)
                middle_distance = calc_distance(feat1_middle, feat_middle)
                large_distance = calc_distance(feat1_large, feat_large)
                result_df.loc[idx,:] = [case_idx, cls, cls, small_distance, middle_distance, large_distance,  file_path]
                
        
        select_can = list(angle_df[(angle_df['class'] != cls) & (angle_df['color'] == select_color)].index)
        random100 = random.sample(select_can, sample_num - select_length)

        for gal_index in random100:
            idx += 1
            gal = angle_df.loc[gal_index]
            car_img = cv2.imread(gal['file_path'])
            feat_small = helper_small.infer(car_img)
            feat_middle = helper_middle.infer(car_img)
            feat_large = helper_large.infer(car_img)
            small_distance = calc_distance(feat1_small, feat_small)
            middle_distance = calc_distance(feat1_middle, feat_middle)
            large_distance = calc_distance(feat1_large, feat_large)
            result_df.loc[idx,:] = [case_idx, cls, gal['class'], small_distance, middle_distance, large_distance,  gal['file_path']]
        num += 1            
        
        print(num)
    
    return result_df

In [ ]:
df = random_df(sample_num=90)
df.to_csv('/content/drive/MyDrive/공유폴더/csv/정면_같은색.csv')
df = random_df(angle = '후면')
df.to_csv('/content/drive/MyDrive/공유폴더/csv/후면_같은색.csv')
df = random_df(angle = '측면')
df.to_csv('/content/drive/MyDrive/공유폴더/csv/측면_같은색.csv')

In [ ]:
angle_df = cropped_img_df[cropped_img_df['angle'] == '후면'].reset_index(drop=True)
angle_df['size'].value_counts()

In [ ]:
angle_df[angle_df['size']=='경차']['class'].value_counts()

In [ ]:
import random
import pandas as pd
import random
import cv2

def random_df(angle = '정면', sample_num=100, path_df = cropped_img_df, unique_df = unique_df):
    
    path_dict = {'small':'/content/drive/MyDrive/공유폴더/fast-reid/outputs/VehicleID_Baseline/baseline_R50.onnx',
                 'middle':'/content/drive/MyDrive/공유폴더/fast-reid/outputs/VeRi_Baseline/baseline_R50.onnx',
                 'large':'/content/drive/MyDrive/공유폴더/fast-reid/outputs/VERI_Wild_Baseline/baseline_R50.onnx'
                 }
    
    idx = 0
    result_df = pd.DataFrame(columns = ['case', 'target_model', 'car_model', 'small_distance', 'middle_distance', 'large_distance', 'file_path', 'same'])
    case_idx = 0
    num = 0

    angle_df = path_df[path_df['angle'] == angle].reset_index(drop=True)
    path_v = angle_df['class'].value_counts()
    cls_over5= path_v[path_v>=5].keys()
    
    helper_small = ReidHelper(path_dict['small'])
    helper_middle = ReidHelper(path_dict['middle'])
    helper_large = ReidHelper(path_dict['large'])
    

    for cls in cls_over5:
        select_path = angle_df[angle_df['class']== cls]['file_path'].values
        select_length = len(select_path)
        select_color = angle_df[angle_df['class']== cls]['color'].values[0]

        feat1_small = None
        feat1_middle = None
        feat1_large = None

        for index, file_path in enumerate(select_path):
            car_img = cv2.imread(file_path)
            feat_small = helper_small.infer(car_img)
            feat_middle = helper_middle.infer(car_img)
            feat_large = helper_large.infer(car_img)

            if index == 0:
                idx +=1
                feat1_small = feat_small
                feat1_middle =feat_middle
                feat1_large = feat_large
                ex = angle_df[angle_df['class']==cls].iloc[0]
                result_df.loc[idx,:] = [case_idx, cls, cls, 0, 0, 0,  file_path, 0]
                
            else:
                idx +=1
                small_distance = calc_distance(feat1_small, feat_small)
                middle_distance = calc_distance(feat1_middle, feat_middle)
                large_distance = calc_distance(feat1_large, feat_large)
                result_df.loc[idx,:] = [case_idx, cls, cls, small_distance, middle_distance, large_distance,  file_path, 0]
                
        
        select_can = list(angle_df[(angle_df['class'] != cls) & (angle_df['color'] != select_color)].index)
        random100 = random.sample(select_can, sample_num - select_length)

        for gal_index in random100:
            idx += 1
            gal = angle_df.loc[gal_index]
            car_img = cv2.imread(gal['file_path'])
            feat_small = helper_small.infer(car_img)
            feat_middle = helper_middle.infer(car_img)
            feat_large = helper_large.infer(car_img)
            small_distance = calc_distance(feat1_small, feat_small)
            middle_distance = calc_distance(feat1_middle, feat_middle)
            large_distance = calc_distance(feat1_large, feat_large)
            result_df.loc[idx,:] = [case_idx, cls, gal['class'], small_distance, middle_distance, large_distance,  gal['file_path'],0]
        num += 1            
        case_idx +=1

        print(num)

    
    result_df.loc[result_df[result_df['target_model'] == result_df['car_model']].index, 'same'] = 1
    
    return result_df

In [ ]:
df = random_df(sample_num=90)
df.to_csv('/content/drive/MyDrive/공유폴더/csv/정면_다른색.csv')
df = random_df(angle = '후면')
df.to_csv('/content/drive/MyDrive/공유폴더/csv/후면_다른색.csv')
df = random_df(angle = '측면')
df.to_csv('/content/drive/MyDrive/공유폴더/csv/측면_다른색.csv')

In [ ]:
angle_df = cropped_img_df[cropped_img_df['angle'] == '정면'].reset_index(drop=True)
angle_df['clsss']

In [ ]:
import random
import pandas as pd
import random
import cv2

def random_df(angle = '정면', standard = None, same = True, sample_num=100, path_df = cropped_img_df, unique_df = unique_df):
    
    path_dict = {'small':'/content/drive/MyDrive/공유폴더/fast-reid/outputs/VehicleID_Baseline/baseline_R50.onnx',
                 'middle':'/content/drive/MyDrive/공유폴더/fast-reid/outputs/VeRi_Baseline/baseline_R50.onnx',
                 'large':'/content/drive/MyDrive/공유폴더/fast-reid/outputs/VERI_Wild_Baseline/baseline_R50.onnx'
                 }
    
    idx = 0
    result_df = pd.DataFrame(columns = ['case', 'target_model', 'car_model', 'small_distance', 'middle_distance', 'large_distance', 'file_path', 'same'])
    case_idx = 0

    angle_df = path_df[path_df['angle'] == angle].reset_index(drop=True)
    path_v = angle_df['class'].value_counts()
    cls_over5= path_v[path_v>=5].keys()
    cls_over5= random.sample(list(path_v[path_v>=5].keys()),100)
    
    helper_small = ReidHelper(path_dict['small'])
    helper_middle = ReidHelper(path_dict['middle'])
    helper_large = ReidHelper(path_dict['large'])
    

    for cls in cls_over5:
        select_path = angle_df[angle_df['class']== cls]['file_path'].values
        select_length = len(select_path)
        if standard != None:
            select_standard = angle_df[angle_df['class']== cls][standard].values[0]

        feat1_small = None
        feat1_middle = None
        feat1_large = None

        for index, file_path in enumerate(select_path):
            car_img = cv2.imread(file_path)
            feat_small = helper_small.infer(car_img)
            feat_middle = helper_middle.infer(car_img)
            feat_large = helper_large.infer(car_img)

            if index == 0:
                idx +=1
                feat1_small = feat_small
                feat1_middle =feat_middle
                feat1_large = feat_large
                ex = angle_df[angle_df['class']==cls].iloc[0]
                result_df.loc[idx,:] = [case_idx, cls, cls, 0, 0, 0,  file_path, 0]
                
            else:
                idx +=1
                small_distance = calc_distance(feat1_small, feat_small)
                middle_distance = calc_distance(feat1_middle, feat_middle)
                large_distance = calc_distance(feat1_large, feat_large)
                result_df.loc[idx,:] = [case_idx, cls, cls, small_distance, middle_distance, large_distance,  file_path, 0]
                
        
        if same:
            select_can = list(angle_df[(angle_df['class'] != cls) & (angle_df[standard] == select_standard)].index)
        else:
            select_can = list(angle_df[(angle_df['class'] != cls) & (angle_df[standard] != select_standard)].index)

        random100 = random.sample(select_can, sample_num - select_length)

        for gal_index in random100:
            idx += 1
            gal = angle_df.loc[gal_index]
            car_img = cv2.imread(gal['file_path'])
            feat_small = helper_small.infer(car_img)
            feat_middle = helper_middle.infer(car_img)
            feat_large = helper_large.infer(car_img)
            small_distance = calc_distance(feat1_small, feat_small)
            middle_distance = calc_distance(feat1_middle, feat_middle)
            large_distance = calc_distance(feat1_large, feat_large)
            result_df.loc[idx,:] = [case_idx, cls, gal['class'], small_distance, middle_distance, large_distance,  gal['file_path'],0]        
        case_idx +=1

        print(case_idx)

    
    result_df.loc[result_df[result_df['target_model'] == result_df['car_model']].index, 'same'] = 1
    
    return result_df

In [ ]:
df = random_df(standard = 'size')
df.to_csv('/content/drive/MyDrive/공유폴더/csv/정면_같은크기.csv')
df = random_df(angle = '후면',standard = 'size')
df.to_csv('/content/drive/MyDrive/공유폴더/csv/후면_같은크기.csv')
df = random_df(angle = '측면',standard = 'size')
df.to_csv('/content/drive/MyDrive/공유폴더/csv/측면_같은크기.csv')

In [ ]:
df = random_df(standard = 'size',same=False)
df.to_csv('/content/drive/MyDrive/공유폴더/csv/정면_다른크기.csv')
df = random_df(angle = '후면',standard = 'size',same=False)
df.to_csv('/content/drive/MyDrive/공유폴더/csv/후면_다른크기.csv')
df = random_df(angle = '측면',standard = 'size',same=False)
df.to_csv('/content/drive/MyDrive/공유폴더/csv/측면_다른크기.csv')

In [ ]:
import random
import pandas as pd
import random
import cv2

def random_df(angle = '정면', standard = None, same = True, sample_num=100, path_df = cropped_img_df, unique_df = unique_df):
    
    path_dict = {'small':'/content/drive/MyDrive/공유폴더/fast-reid/outputs/VeRi_Baseline/baseline_R50.onnx',
                'middle':'/content/drive/MyDrive/공유폴더/fast-reid/outputs/VehicleID_Baseline/baseline_R50.onnx',
                'large':'/content/drive/MyDrive/공유폴더/fast-reid/outputs/VERI_Wild_Baseline/baseline_R50.onnx'
                 }
    
    idx = 0
    result_df = pd.DataFrame(columns = ['case', 'target_model', 'car_model', 'small_distance', 'middle_distance', 'large_distance', 'file_path', 'same'])
    case_idx = 0

    angle_df = path_df[path_df['angle'] == angle].reset_index(drop=True)
    path_v = angle_df['class'].value_counts()
    cls_over5= path_v[path_v>=5].keys()
    cls_over5= random.sample(list(path_v[path_v>=5].keys()),100)
    
    helper_small = ReidHelper(path_dict['small'])
    helper_middle = ReidHelper(path_dict['middle'])
    helper_large = ReidHelper(path_dict['large'])
    

    for cls in cls_over5:
        select_path = angle_df[angle_df['class']== cls]['file_path'].values
        select_path = random.sample(list(angle_df[angle_df['class']== cls]['file_path'].values), 5)
        if standard != None:
            select_standard = angle_df[angle_df['class']== cls][standard].values[0]

        feat1_small = None
        feat1_middle = None
        feat1_large = None

        for index, file_path in enumerate(select_path):
            car_img = cv2.imread(file_path)
            feat_small = helper_small.infer(car_img)
            feat_middle = helper_middle.infer(car_img)
            feat_large = helper_large.infer(car_img)

            if index == 0:
                idx +=1
                feat1_small = feat_small
                feat1_middle =feat_middle
                feat1_large = feat_large
                ex = angle_df[angle_df['class']==cls].iloc[0]
                result_df.loc[idx,:] = [case_idx, cls, cls, 0, 0, 0,  file_path, 0]
                
            else:
                idx +=1
                small_distance = calc_distance(feat1_small, feat_small)
                middle_distance = calc_distance(feat1_middle, feat_middle)
                large_distance = calc_distance(feat1_large, feat_large)
                result_df.loc[idx,:] = [case_idx, cls, cls, small_distance, middle_distance, large_distance,  file_path, 0]
                
        if standard:
            if same:
                select_can = list(angle_df[(angle_df['class'] != cls) & (angle_df[standard] == select_standard)].index)
            else:
                select_can = list(angle_df[(angle_df['class'] != cls) & (angle_df[standard] != select_standard)].index)
        
        else:
            select_can = list(angle_df[angle_df['class'] != cls].index)
        
        random100 = random.sample(select_can, sample_num - 5)

        for gal_index in random100:
            idx += 1
            gal = angle_df.loc[gal_index]
            car_img = cv2.imread(gal['file_path'])
            feat_small = helper_small.infer(car_img)
            feat_middle = helper_middle.infer(car_img)
            feat_large = helper_large.infer(car_img)
            small_distance = calc_distance(feat1_small, feat_small)
            middle_distance = calc_distance(feat1_middle, feat_middle)
            large_distance = calc_distance(feat1_large, feat_large)
            result_df.loc[idx,:] = [case_idx, cls, gal['class'], small_distance, middle_distance, large_distance,  gal['file_path'],0]        
        case_idx +=1

        print(case_idx)

    
    result_df.loc[result_df[result_df['target_model'] == result_df['car_model']].index, 'same'] = 1
    
    return result_df

In [ ]:
df = random_df()
df.to_csv('/content/drive/MyDrive/공유폴더/csv/진짜비교/정면.csv')
df = random_df(angle = '후면')
df.to_csv('/content/drive/MyDrive/공유폴더/csv/진짜비교/후면.csv')
df = random_df(angle = '측면')
df.to_csv('/content/drive/MyDrive/공유폴더/csv/진짜비교/측면.csv')

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100


In [ ]:
df = random_df(standard = 'size')
df.to_csv('/content/drive/MyDrive/공유폴더/csv/진짜비교/정면_같은크기.csv')
df = random_df(angle = '후면',standard = 'size')
df.to_csv('/content/drive/MyDrive/공유폴더/csv/진짜비교/후면_같은크기.csv')
df = random_df(angle = '측면',standard = 'size')
df.to_csv('/content/drive/MyDrive/공유폴더/csv/진짜비교/측면_같은크기.csv')

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100


In [ ]:
df = random_df(standard = 'size', same= False)
df.to_csv('/content/drive/MyDrive/공유폴더/csv/진짜비교/정면_다른크기.csv')
df = random_df(angle = '후면',standard = 'size', same= False)
df.to_csv('/content/drive/MyDrive/공유폴더/csv/진짜비교/후면_다른크기.csv')
df = random_df(angle = '측면',standard = 'size', same= False)
df.to_csv('/content/drive/MyDrive/공유폴더/csv/진짜비교/측면_다른크기.csv')

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100


In [ ]:
df = random_df(standard = 'color',same= False)
df.to_csv('/content/drive/MyDrive/공유폴더/csv/진짜비교/정면_다른색상.csv')
df = random_df(angle = '후면',standard = 'color' ,same= False)
df.to_csv('/content/drive/MyDrive/공유폴더/csv/진짜비교/후면_다른색상.csv')
df = random_df(angle = '측면',standard = 'color' ,same= False)
df.to_csv('/content/drive/MyDrive/공유폴더/csv/진짜비교/측면_다른색상.csv')

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100


In [ ]:
import random
import pandas as pd
import random
import cv2

def random_df(angle = '정면', standard = None, same = True, sample_num=100, path_df = cropped_img_df, unique_df = unique_df):
    
    path_dict = {'small':'/content/drive/MyDrive/공유폴더/fast-reid/outputs/VeRi_Baseline/baseline_R50.onnx',
                'middle':'/content/drive/MyDrive/공유폴더/fast-reid/outputs/VehicleID_Baseline/baseline_R50.onnx',
                'large':'/content/drive/MyDrive/공유폴더/fast-reid/outputs/VERI_Wild_Baseline/baseline_R50.onnx'
                 }
    
    idx = 0
    result_df = pd.DataFrame(columns = ['case', 'target_model', 'car_model', 'small_distance', 'middle_distance', 'large_distance', 'file_path', 'same'])
    case_idx = 0

    angle_df = path_df[(path_df['angle'] == angle) & (path_df['color'] != '빨강')].reset_index(drop=True)
    path_v = angle_df['class'].value_counts()
    cls_over5= path_v[path_v>=5].keys()
    cls_over5= random.sample(list(path_v[path_v>=5].keys()),100)
    
    helper_small = ReidHelper(path_dict['small'])
    helper_middle = ReidHelper(path_dict['middle'])
    helper_large = ReidHelper(path_dict['large'])
    

    for cls in cls_over5:
        select_path = angle_df[angle_df['class']== cls]['file_path'].values
        select_path = random.sample(list(angle_df[angle_df['class']== cls]['file_path'].values), 5)
        if standard != None:
            select_standard = angle_df[angle_df['class']== cls][standard].values[0]

        feat1_small = None
        feat1_middle = None
        feat1_large = None

        for index, file_path in enumerate(select_path):
            car_img = cv2.imread(file_path)
            feat_small = helper_small.infer(car_img)
            feat_middle = helper_middle.infer(car_img)
            feat_large = helper_large.infer(car_img)

            if index == 0:
                idx +=1
                feat1_small = feat_small
                feat1_middle =feat_middle
                feat1_large = feat_large
                ex = angle_df[angle_df['class']==cls].iloc[0]
                result_df.loc[idx,:] = [case_idx, cls, cls, 0, 0, 0,  file_path, 0]
                
            else:
                idx +=1
                small_distance = calc_distance(feat1_small, feat_small)
                middle_distance = calc_distance(feat1_middle, feat_middle)
                large_distance = calc_distance(feat1_large, feat_large)
                result_df.loc[idx,:] = [case_idx, cls, cls, small_distance, middle_distance, large_distance,  file_path, 0]
                
        if standard:
            if same:
                select_can = list(angle_df[(angle_df['class'] != cls) & (angle_df[standard] == select_standard)].index)
            else:
                select_can = list(angle_df[(angle_df['class'] != cls) & (angle_df[standard] != select_standard)].index)
        
        else:
            select_can = list(angle_df[angle_df['class'] != cls].index)
        
        random100 = random.sample(select_can, sample_num - 5)

        for gal_index in random100:
            idx += 1
            gal = angle_df.loc[gal_index]
            car_img = cv2.imread(gal['file_path'])
            feat_small = helper_small.infer(car_img)
            feat_middle = helper_middle.infer(car_img)
            feat_large = helper_large.infer(car_img)
            small_distance = calc_distance(feat1_small, feat_small)
            middle_distance = calc_distance(feat1_middle, feat_middle)
            large_distance = calc_distance(feat1_large, feat_large)
            result_df.loc[idx,:] = [case_idx, cls, gal['class'], small_distance, middle_distance, large_distance,  gal['file_path'],0]        
        case_idx +=1

        print(case_idx)

    
    result_df.loc[result_df[result_df['target_model'] == result_df['car_model']].index, 'same'] = 1
    
    return result_df

In [ ]:
df = random_df(standard = 'color')
df.to_csv('/content/drive/MyDrive/공유폴더/csv/진짜비교/정면_같은색상.csv')
df = random_df(angle = '후면',standard = 'color')
df.to_csv('/content/drive/MyDrive/공유폴더/csv/진짜비교/후면_같은색상.csv')
df = random_df(angle = '측면',standard = 'color')
df.to_csv('/content/drive/MyDrive/공유폴더/csv/진짜비교/측면_같은색상.csv')

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
